In [1]:
from dotenv import load_dotenv

load_dotenv()

True

In [2]:
from dataclasses import dataclass
from langchain.agents.middleware import dynamic_prompt, ModelRequest

@dataclass
class LanguageContext:
    user_language: str = "English"

@dynamic_prompt
def user_language_prompt(request: ModelRequest) -> str:
    """Generate system prompt based on user role."""
    user_language = request.runtime.context.user_language
    base_prompt = "You are a helpful assistant."

    if user_language != "English":
        return f"{base_prompt} only respond in {user_language}."
    elif user_language == "English":
        return base_prompt

In [8]:
from langchain.agents import create_agent
from langchain_aws import ChatBedrock

# 1. CONFIGURACIÓN PARA DEEPSEEK-R1 (Razonamiento Complejo)
# Ideal para agentes que necesitan planificar pasos lógicos.
# llm = ChatBedrock(
#     model_id="us.deepseek.r1-v1:0",  # ID oficial validado
#     region_name="us-east-1",
#     model_kwargs={
#         "temperature": 0.6, # DeepSeek recomienda 0.6 para razonamiento
#         "max_tokens": 8192,  # Recomendado para no degradar calidad del CoT
#         "top_p": 0.95,
#     }
# )


# llm = ChatBedrock(
#     model_id="us.deepseek.v3-v1:0", # Prueba este primero
#     region_name="us-east-1",        # O us-west-2
#     model_kwargs={
#         "temperature": 0.7,
#         "max_tokens": 4096
#     }
# )
from langchain_aws import ChatBedrock
llm = ChatBedrock(
model_id="us.meta.llama4-scout-17b-instruct-v1:0",  # Nota el prefijo "us."
# model_id="cohere.command-r-plus-v1:0",
region_name="us-east-1",
model_kwargs={
"temperature": 0.5,
"max_tokens": 2048,
"top_p": 0.9,
}
)


# llm = ChatBedrock(
#     model_id="us.meta.llama4-maverick-17b-instruct-v1:0",  # Nota el prefijo "us."
#     region_name="us-east-1",
#     model_kwargs={
#         "temperature": 0.5,
#         "max_tokens": 2048,
#         "top_p": 0.9,
#     }
# )

# llm = ChatBedrock(
#     model_id="amazon.nova-lite-v1:0",  # Nota el prefijo "us."
#     region_name="us-east-1",
#     model_kwargs={
#         "temperature": 0.5,
#         "max_tokens": 2048,
#         "top_p": 0.9,
#     }
# )

In [9]:
from langchain.agents import create_agent

agent = create_agent(
    model=llm,
    context_schema=LanguageContext,
    middleware=[user_language_prompt]
)

In [10]:
from langchain.messages import HumanMessage

response = agent.invoke(
    {"message": [HumanMessage(content="Hello, how are you?")]},
    context=LanguageContext(user_language="Irish")
)

print(response["messages"][-1].content)



Táim réidh chun cabhrú! Conas is féidir liom tú a chur ar an eolas?


In [11]:
from langchain.messages import HumanMessage

response = agent.invoke(
    {"message": [HumanMessage(content="Hello, how are you?")]},
    context=LanguageContext(user_language="Spanish")
)

print(response["messages"][-1].content)



¡Claro! Estoy aquí para ayudarte. ¿En qué puedo asistirte?


In [12]:
from langchain.messages import HumanMessage

response = agent.invoke(
    {"message": [HumanMessage(content="Hello, how are you?")]},
    context=LanguageContext(user_language="French")
)

print(response["messages"][-1].content)



Bien sûr ! Comment puis-je vous aider aujourd'hui ?
